## ライブラリ

In [74]:
%matplotlib inline

# Basic
import os
import random
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from IPython.display import Image, display_png
import collections
import category_encoders as ce

# Sampling or Devison of Data
#from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

# Machine Learning
from sklearn import preprocessing, metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, SVR

# Parameter Turning
from sklearn.model_selection import GridSearchCV

# GBDT
import xgboost as xgb
import catboost
import lightgbm as lgb

# SageMarker
import io, boto3
from sagemaker import get_execution_role

# Warning
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [75]:
# レコード数とカラム数が多い場合に設定する
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# pandasのフォーマットを変更
pd.options.display.precision = 3
pd.options.display.float_format = '{:.2f}'.format

---

## 関数

In [76]:
# Confusion Matrix グラフ
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    #for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    for i, j in ((x,y) for x in range(cm.shape[0]) for y in range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [77]:
# True/Pred 棒グラフ
def plot_true_pred_bar(d_true, d_pred):

    plt.figure(figsize=(8, 6))
    plt.bar(np.array([x for x in collections.Counter(d_true)]) +0.2, np.array([x for x in collections.Counter(d_true).values()]), color="#CBDEF0", width=0.4, label="True")
    plt.bar(np.array([x for x in collections.Counter(d_pred)]) -0.2, np.array([x for x in collections.Counter(d_pred).values()]), color="#60A6D1", width=0.4, label="Pred")

    plt.xticks(range(0,9))
    plt.legend(loc=2)
    plt.show()

In [78]:
# 型変換関数
def reduce_memory(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df

---

## データ読み込み

In [79]:
# データ定義
dtype_dm = {
    "推薦ID":"int32",
    "求人ID":"int32",
    "契約世代ID":"int32",
    "求職者子CD":"int32",
    "決定年収":"int32",
    "入社FLG":"int8",
    "受注額_金額_合計":"int32",
    "採用人数":"int16",
    "転職決定時年齢":"int8",
}

dtype_ii = {
    "推薦ID":"int32",
    "FLAG_一割以上上":"int8",
    "FLAG_一割以上下":"int8",
    "FLAG_一円でも上":"int8",
    "FLAG_一円でも下":"int8",
    "FLAG_完全一致":"int8",
    "前年収_採用年収":"int32",
    "後年収":"int32",
    "年齢":"int16",
}

In [80]:
# AWSのrole確認
role = get_execution_role()
role

'arn:aws:iam::707539689644:role/service-role/AmazonSageMaker-ExecutionRole-20190401T165949'

In [81]:
# S3接続
s3_client = boto3.client('s3')
data_bucket_name='rc-datum-s3'

obj_list=s3_client.list_objects(Bucket=data_bucket_name)
file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
print(file)

['ACD.tsv', 'datamart.txt', 'index_inspect.tsv', 'test_ok_feature.tsv']


In [82]:
response = s3_client.get_object(Bucket=data_bucket_name, Key=file[1])
response_body = response["Body"].read()
df_dm = pd.read_table(io.BytesIO(response_body), dtype=dtype_dm, low_memory=False) 

print(df_dm.shape)
#display(df_dm.head())

(419128, 68)


In [83]:
response = s3_client.get_object(Bucket=data_bucket_name, Key=file[2])
response_body = response["Body"].read()
df_ii = pd.read_table(io.BytesIO(response_body), dtype=dtype_ii, low_memory=False) 

print(df_ii.shape)
#display(df_ii.head())

(381295, 67)


In [84]:
response = s3_client.get_object(Bucket=data_bucket_name, Key=file[0])
response_body = response["Body"].read()
df_acd = pd.read_table(io.BytesIO(response_body), low_memory=False) 
df_acd = reduce_memory(df_acd)

print(df_acd.shape)
#display(df_acd.head())

Initial df memory usage is 40.09 MB for 9 columns
Final memory usage is: 33.41 MB - decreased by 16.7%
(583803, 9)


#### カラム削除

In [85]:
# datamart
df_dm.drop(["求人ID","契約世代ID","求職者子CD"], axis=1, inplace=True)
# 代表職種と経験職種名／求人職種コードは中身がほぼ同じなので、代表職種のみ残す
df_dm.drop(["経験職種名","求人職種コード"], axis=1, inplace=True)
df_dm.drop(["資格2","資格3","資格4","資格5","決定計上日","求人雇用形態","経験業界名2","経験業界名3","経験業界名4","経験業界名5"], axis=1, inplace=True)
df_dm.drop(["求人先_法人CD","求人先_窓口CD","求人業界2","求人業界3","求人業界4","求人業界5"], axis=1, inplace=True)
df_dm.drop(["経験業界名1(2008年8月以前取得用)","経験職種名1(代替取得方法)","経験職種名2(代替取得方法)","経験職種名3(代替取得方法)","経験職種名4(代替取得方法)","経験職種名5(代替取得方法)","求人職種(代替取得方法)",], axis=1, inplace=True)
# カラム数が増えるので市区郡は一旦削除する
df_dm.drop(["住所_市区郡"], axis=1, inplace=True)
print(df_dm.shape)
#df_dm.head()

# index_inspect
df_ii.drop(["FLAG_一割以上上","FLAG_一割以上下","FLAG_一円でも上","FLAG_一円でも下","FLAG_完全一致","前年収_X","前年収_Y","前年収_A","前年収_B","前年収_C","転職後_企業コード"], axis=1, inplace=True)
df_ii.drop(["統合法人C","顧客Ｃ","帝国Ｃ"], axis=1, inplace=True)
# 従業員数と資本金は明らかに数値がおかしいので、RC社から返答あるまで一旦削除しとく
df_ii.drop(["従業員数＿最新","資本金＿最新"], axis=1, inplace=True)
df_ii.drop(["決算ＹＭ＿前回","従業員数＿前回","資本金＿前回","年商＿前回","利益額＿前回","申告所得額＿前回","年商伸率＿前回","利益伸率＿前回","評点＿前回","決算ＹＭ＿前々回","従業員数＿前々回","資本金＿前々回","年商＿前々回","利益額＿前々回","申告所得額＿前々回","年商伸率＿前々回","利益伸率＿前々回","評点＿前々回",], axis=1, inplace=True)
# 学歴と性別はdatamartの方を使う
df_ii.drop(["性別","学歴"], axis=1, inplace=True)
print(df_ii.shape)
#df_ii.head()

# ACD
df_acd.drop(["卒業中退区分"], axis=1, inplace=True)
print(df_acd.shape)
#df_acd.head()

# ACDは重複データがやたらと多いので、dropnaして減らす
df_acd_2 = df_acd.dropna()
# 重複行がたくさんあるので、最初だけ残してあとは削除する
df_acd_2 = df_acd_2.drop_duplicates("求職者親CD", keep="first")

df_acd_2.shape

(419128, 39)
(381295, 31)
(583803, 8)


(112262, 8)

### データ結合

In [86]:
df_ = pd.merge(df_dm, df_ii, on="推薦ID", how="left")
print(df_.shape)
#df_.head()

df_2 = pd.merge(df_, df_acd_2, on="求職者親CD", how="left")
print(df_2.shape)
#df_2.head()

#df_3 = df_2.copy()

# 推薦IDと求職者親CDを削除する
df_.drop(["推薦ID","求職者親CD"], axis=1, inplace=True)
df_3 = df_.copy()

(419128, 69)
(419128, 76)


---

## 前処理

In [87]:
# 前年収と後年収の異常値を丸める
df_3.loc[df_3["前年収_採用年収"]>=5000, "前年収_採用年収"] = 5000
df_3.loc[df_3["後年収"]>=50000000, "後年収"] = 50000000

diff = df_3["後年収"] - (df_3["前年収_採用年収"] * 10000)
#df_cs2["diff_cate"] = pd.cut(diff, [-46563800, -1500000, -1000000, -500000, 0 , 500000, 1000000, 1500000, 2000000, 45000000])
df_3["diff_cate"] = pd.cut(diff, [-46563800, -500000, 0 , 500000, 1000000, 1500000, 2000000, 45000000])

# カテゴリ区分リスト
#cate_name = ["0: -150以下" ,"1: -150 ～ -100", "2: -100 ～ -50", "3: -50 ～ 0", "4: 0 ～ 50", "5: 50 ～ 100", "6: 100 ～ 150", "7: 150 ～ 200", "8: 200以上"]
cate_name = ["0: -50以下" ,"1: -50 ～ 0", "2: 0 ～ 50", "3: 50 ～ 100", "4: 100 ～ 150", "5: 150 ～ 200", "6: 200以上"]

In [88]:
# null件数の多い変数の穴埋め処理
# これやらないでdropnaすると件数が激減する
df_3.loc[df_3["雇用形態"].isnull(), "雇用形態"] = "なし"
df_3.loc[df_3["代表職種"].isnull(), "代表職種"] = "なし"
df_3.loc[df_3["経験業界名1"].isnull(), "経験業界名1"] = "なし"
df_3.loc[df_3["職務クラス"].isnull(), "職務クラス"] = "なし"
df_3.loc[df_3["語学"].isnull(), "語学"] = "なし"
df_3.loc[df_3["語学レベル_3段階"].isnull(), "語学レベル_3段階"] = "なし"
df_3.loc[df_3["資格1"].isnull(), "資格1"] = "なし"
df_3.loc[df_3["外資資本比率"].isnull(), "外資資本比率"] = -1
df_3.loc[df_3["求人語学"].isnull(), "求人語学"] = "なし"
df_3.loc[df_3["求人語学レベル"].isnull(), "求人語学レベル"] = "なし"
df_3.loc[df_3["年間賞与"].isnull(), "年間賞与"] = "なし"
df_3.loc[df_3["残業手当"].isnull(), "残業手当"] = "なし"
df_3.loc[df_3["住宅手当"].isnull(), "住宅手当"] = "なし"
df_3.loc[df_3["職種_転職前_小分類"].isnull(), "職種_転職前_小分類"] = "なし"
df_3.loc[df_3["職種_転職前_大分類"].isnull(), "職種_転職前_大分類"] = "なし"
df_3.loc[df_3["職種_転職後_小分類"].isnull(), "職種_転職後_小分類"] = "なし"
df_3.loc[df_3["職種_転職後_大分類"].isnull(), "職種_転職後_大分類"] = "なし"
df_3.loc[df_3["業種_転職前_小分類"].isnull(), "業種_転職前_小分類"] = "なし"
df_3.loc[df_3["業種_転職前_大分類"].isnull(), "業種_転職前_大分類"] = "なし"
df_3.loc[df_3["業種_転職後_小分類"].isnull(), "業種_転職後_小分類"] = "なし"
df_3.loc[df_3["業種_転職後_大分類"].isnull(), "業種_転職後_大分類"] = "なし"

# 転職後企業名を各企業の平均年収に変更する
df_company_income = pd.DataFrame(index=[i for i in range(0,len(df_3.groupby("転職後_企業名").mean()["後年収"]))])
df_company_income["転職後_企業名"] = pd.DataFrame((df_3.groupby("転職後_企業名").mean()["後年収"]/10000).index)
df_company_income["後年収"] = pd.DataFrame((df_3.groupby("転職後_企業名").mean()["後年収"]/10000).values)
#df_company_income = pd.DataFrame(df_3.groupby("転職後_企業名").mean()["後年収"]/10000)

df_4 = pd.merge(df_3, df_company_income, on="転職後_企業名", how="left")
df_4 = df_4.rename(columns={"後年収_x":"後年収", "後年収_y":"転職後_企業平均年収"})
df_4.drop("転職後_企業名", axis=1, inplace=True)
#df_cs4.shape

# 「年齢」と「年齢_5歳刻み」は情報としてかぶっているので「年齢」を削除する
#df_4.drop(["年齢_5歳刻み"], axis=1, inplace=True)
df_4.drop(["年齢"], axis=1, inplace=True)

In [89]:
# 中央値計算
df_before_jobtype_income = pd.DataFrame(index=[i for i in range(0,len(df_3.groupby("職種_転職前_小分類").mean()["後年収"]))])
df_before_jobtype_income["職種_転職前_小分類"] = pd.DataFrame((df_3.groupby("職種_転職前_小分類").mean()["後年収"]/10000).index)
df_before_jobtype_income["後年収"] = pd.DataFrame((df_3.groupby("職種_転職前_小分類").mean()["後年収"]/10000).values)

df_after_jobtype_income   = pd.DataFrame(df_4.groupby("職種_転職後_小分類").median()["後年収"]/10000)
df_after_jobtype_income["職種_転職後_小分類"] = pd.DataFrame((df_3.groupby("職種_転職後_小分類").mean()["後年収"]/10000).index)
df_after_jobtype_income["後年収"] = pd.DataFrame((df_3.groupby("職種_転職後_小分類").mean()["後年収"]/10000).values)

df_before_industry_income = pd.DataFrame(df_4.groupby("業種_転職前_小分類").median()["後年収"]/10000)
df_before_industry_income["業種_転職前_小分類"] = pd.DataFrame((df_3.groupby("業種_転職前_小分類").mean()["後年収"]/10000).index)
df_before_industry_income["後年収"] = pd.DataFrame((df_3.groupby("業種_転職前_小分類").mean()["後年収"]/10000).values)

df_after_industry_income  = pd.DataFrame(df_4.groupby("業種_転職後_小分類").median()["後年収"]/10000)
df_after_industry_income["業種_転職後_小分類"] = pd.DataFrame((df_3.groupby("業種_転職後_小分類").mean()["後年収"]/10000).index)
df_after_industry_income["後年収"] = pd.DataFrame((df_3.groupby("業種_転職後_小分類").mean()["後年収"]/10000).values)

# データ結合
df_5 = pd.merge(df_4, df_before_jobtype_income,  on="職種_転職前_小分類", how="left", suffixes=["", "_1"])
df_5 = pd.merge(df_5, df_after_jobtype_income,   on="職種_転職後_小分類", how="left", suffixes=["", "_1"])
df_5 = pd.merge(df_5, df_before_industry_income, on="業種_転職前_小分類", how="left", suffixes=["", "_2"])
df_5 = pd.merge(df_5, df_after_industry_income,  on="業種_転職後_小分類", how="left", suffixes=["", "_2"])

# カラム名を変更
df_5 = df_5.rename(columns={
    "前年収_採用年収_1":"職種_転職前_小分類_平均年収",
    "後年収_1":"職種_転職後_小分類_平均年収",
    "前年収_採用年収_2":"業種_転職前_小分類_平均年収",
    "後年収_2":"業種_転職後_小分類_平均年収"
})

# 小分類データを削除
df_5.drop(["職種_転職前_小分類","職種_転職後_小分類","業種_転職前_小分類","業種_転職後_小分類"], axis=1, inplace=True)
#df_5.head()

In [91]:
# 「後年収」を削除
df_5.drop(["後年収"], axis=1, inplace=True)

# NULL値を削除
#df_6 = df_5.dropna()
#df_6.shape

---

## データセット

In [92]:
cat_features_cols = [
    "求職者状態",
    "サービスステータス",
    "アクセス目的",
    "アクセス手段",
    "認知媒体",
    "性別",
    "都道府県名称",
    "最終学歴",
    "現職",
    "雇用形態",
    "代表職種",
    "経験業界名1",
    "職務クラス",
    "語学",
    "語学レベル_3段階",
    "資格1",
    "総合職FLG",
    "求人語学",
    "求人語学レベル",
    "住所_都道府県",
    "勤務地",
    "求人業界1",
    "職種_転職前_大分類",
    "職種_転職後_大分類",
    "業種_転職前_大分類",
    "業種_転職後_大分類",
    "年齢_5歳刻み",
    "証券Ｃ",
    "上場区分Ｃ",
    "上場区分名",
    "メイン業種分類Ｃ",
]

In [93]:
# ダミー変数化
#df_ = pd.get_dummies(df_6.drop(["diff_cate"], axis=1))
ce_ohe = ce.OneHotEncoder(cols=cat_features_cols, handle_unknown="impute")
df_ = ce_ohe.fit_transform(df_5.drop(["diff_cate"], axis=1))
df_["diff_cate"] = df_5["diff_cate"]

# 最終データとして設定
df_fin1 = df_.copy()
print(df_fin1.shape)

MemoryError: 